<a href="https://colab.research.google.com/github/meluxis/Analysis_Transformer_ComputationalCost_Optimization/blob/main/BaselineModelT5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Connect google colab and Google Cloud bucket for storage
#Link google account
from google.colab import auth
auth.authenticate_user()

In [6]:
#Link bucket content
BUCKET_NAME = "data_bucket_experimental_analysis"
# Validate bucket
!gsutil ls -b gs://$BUCKET_NAME || gsutil mb -l us-central1 gs://$BUCKET_NAME


gs://data_bucket_experimental_analysis/


In [7]:
#install Tensorflow in environment
!pip install tensorflow
import tensorflow as tf
print("Tensorflow version is",tf.__version__)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.9/644.9 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 70.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 88.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 93.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.3/106.3 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 97.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 4.0 MB/s eta 0:00:00
Tensorflow version is 2.19.0


In [8]:
from google.colab import auth
import os
import tensorflow as tf

# Authenticate with Google Cloud
auth.authenticate_user()

# Set up manual GCS access
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "/content/adc.json"

# TPU Setup (if available)
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print("Connected to TPU:", tpu.master())
except ValueError:
    print("No TPU found. Running on CPU/GPU instead.")


No TPU found. Running on CPU/GPU instead.


In [5]:
#!pip install transformers datasets torch torchvision

In [13]:
#import libraries
import torch
import time
import psutil  # For memory usage
import os


import sys

import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset

from transformers import T5Tokenizer, T5ForConditionalGeneration, TrainingArguments, Trainer, T5Model
from datasets import load_dataset
import argparse


In [12]:
#!pip install tf-keras~=2.16
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.2/69.2 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.9/274.9 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.3/231.3 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 344.1/344.1 kB 13.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0


In [14]:
#import dataset

dataset = load_dataset("google/Synthetic-Persona-Chat")

train_data = dataset["train"]
test_data = dataset["test"]

# Afficher quelques exemples
print(train_data["user 1 personas"][0:5])  # Afficher 5 exemples
print(train_data["user 2 personas"][0:5])
print(train_data["Best Generated Conversation"][0:5])


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/3.11k [00:00<?, ?B/s]

Synthetic-Persona-Chat_train.csv:   0%|          | 0.00/15.9M [00:00<?, ?B/s]

Synthetic-Persona-Chat_valid.csv:   0%|          | 0.00/1.78M [00:00<?, ?B/s]

Synthetic-Persona-Chat_test.csv:   0%|          | 0.00/1.72M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/8938 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/968 [00:00<?, ? examples/s]

['I am 32.\nI do not want a job.\nI play video games all day.\nI still live at home with my parents.', 'I am 32.\nI play video games all day.\nI still live at home with my parents.\nI do not want a job.', 'I am 32.\nI play video games all day.\nI still live at home with my parents.\nI do not want a job.\nI eat all take out.', 'I write.\nI work at mcdonald s.\nI watch youtube videos.\nI have brown hair.', 'I am bald.\nI like to swim.\nMy favorite drink is chocolate milk.\nI don t work.']
['My favorite drink is iced coffee.\nI have a black belt in karate.\nI m in a jazz band and play the saxophone.\nI vacation along lake michigan every summer.', 'I have a ford f150.\nI like ford cars.\nMy truck is black.\nI also like ford trucks.\nI own a ford truck.', 'I can recite the movie young frankenstein word for word.\nI like to make my own clothes.\nI am partially blind.\nI do volunteer work for human rights organizations.\nI can do convincing bird calls.', 'I want to move.\nI don t like feeling

In [15]:
#Textes into Tokens
TOKENIZER = T5Tokenizer.from_pretrained("t5-base")

def preprocess_function(examples):  #Tokenize inputs and targets

  #Combine both personas
  inputs = [p1 + p2 for p1,
            p2 in zip(
                examples["user 1 personas"],
                examples["user 2 personas"])
            ]

  #tokensize inputs (persona 1, persona 2)
  model_inputs = TOKENIZER(inputs,
                           max_length=512,
                           truncation=True,
                           padding="max_length"
                           )

  #tokenize targets (best generated conversations)
  with TOKENIZER.as_target_tokenizer():
      # Ensure 'Best Generated Conversation' is a list of strings
    conversations = [str(conv) for conv in examples["Best Generated Conversation"]]
    labels = TOKENIZER(conversations,
                       max_length=512,
                       truncation=True,
                       padding="max_length"
                       )

  model_inputs["labels"] = labels["input_ids"]
  return model_inputs

# Preprocess the dataset
tokenized_train_dataset = train_data.map(preprocess_function,
                                         batched=True,
                                         batch_size=8
                                         )
tokenized_test_dataset = test_data.map(preprocess_function,
                                       batched=True,
                                         batch_size=8)

# Example usage (accessing tokenized data):
print(tokenized_train_dataset[0]["input_ids"])
print(tokenized_train_dataset[0]["labels"])


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Map:   0%|          | 0/8938 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3961: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/968 [00:00<?, ? examples/s]

[27, 183, 3538, 5, 27, 103, 59, 241, 3, 9, 613, 5, 27, 577, 671, 1031, 66, 239, 5, 27, 341, 619, 44, 234, 28, 82, 1362, 5, 7008, 1305, 3281, 19, 3, 867, 26, 1975, 5, 27, 43, 3, 9, 1001, 6782, 16, 3, 4031, 342, 5, 27, 3, 51, 16, 3, 9, 9948, 1928, 11, 577, 8, 3, 7, 9, 226, 32, 6399, 5, 27, 4257, 590, 6957, 2278, 12588, 334, 1248, 5, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [19]:
#Transform Data for compatibility with pytorch

from torch.utils.data import DataLoader

# Create a custom Dataset class
class PersonaChatDataset(Dataset):
    def __init__(self, dataset):
        self.dataset = dataset

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        item = self.dataset[idx]
        return {
            'input_ids': torch.tensor(item['input_ids']),
            'attention_mask': torch.tensor(item['attention_mask']),
            'labels': torch.tensor(item['labels'])
        }

# Create DataLoaders
train_dataset = PersonaChatDataset(tokenized_train_dataset)
test_dataset = PersonaChatDataset(tokenized_test_dataset)

train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=5) # Adjust batch size as needed
test_dataloader = DataLoader(test_dataset, batch_size=8, shuffle=False, num_workers=5) # Adjust batch size as needed

print(len(train_dataloader))

1118


In [10]:
#Define model from https://colab.research.google.com/github/google-research/text-to-text-transfer-transformer/blob/main/notebooks/t5-trivia.ipynb#scrollTo=DZhAd0U_4B_o
# Required to fix Colab flag parsing issue.

sys.argv = sys.argv[:1]

BASE_DIR = "gs://data_bucket_experimental_analysis" #@param { type: "string" }
if not BASE_DIR or BASE_DIR == "gs://":
  raise ValueError("You must enter a BASE_DIR.")
DATA_DIR = os.path.join(BASE_DIR, "data")
MODELS_DIR = os.path.join(BASE_DIR, "models")
ON_CLOUD = True

MODEL_SIZE = "base" #@param["small", "base", "large", "3B", "11B"]
PRETRAINED_DIR = "t5-base"
MODEL_DIR = os.path.join(MODELS_DIR, MODEL_SIZE)


# Set parallelism and batch size to fit on v2-8 TPU (if possible).
# Limit number of checkpoints to fit within 5GB (if possible).
model_parallelism, train_batch_size, keep_checkpoint_max = {
    "small": (1, 256, 16),
    "base": (2, 128, 8),
    "large": (8, 64, 4),
    "3B": (8, 16, 1),
    "11B": (8, 16, 1)}[MODEL_SIZE]

#tf.io.gfile.makedirs(MODEL_DIR)
# The models from our paper are based on the Mesh Tensorflow Transformer.
#Change to T5ForConditionalGeneration for finetuning task
model = T5ForConditionalGeneration.from_pretrained(PRETRAINED_DIR)


In [11]:
import psutil
import time
from fvcore.nn import FlopCountAnalysis
from codecarbon import EmissionsTracker

# Define the optimizer and loss function
optimizer = optim.AdamW(model.parameters(), lr=5e-5) # Adjust learning rate as needed
loss_fn = nn.CrossEntropyLoss()

# Training loop
epochs = 1  # Adjust the number of epochs
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

start_time = time.time()

print(next(model.parameters()).device)

for epoch in range(epochs):
    tracker = EmissionsTracker()
    tracker.start()
    model.train()
    total_loss = 0
    i=0
    for batch in train_dataloader:
        i+=1
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        print(f"Batch {i}/{len(train_dataloader)} - Loss: {loss.item()}")

    avg_loss = total_loss / len(train_dataloader)
    print(f"Epoch {epoch+1}/{epochs}, Loss: {avg_loss}")

    #Evaluate memory usage
    process = psutil.Process(os.getpid())
    memory_info = process.memory_info()
    print(f"Epoch {epoch+1}/{epochs}, Memory Usage: {memory_info.rss / 1024 / 1024} MB")

    #Evaluate training time per step
    total_time = time.time() - start_time
    print(f"Epoch {epoch+1}/{epochs}, Total Training Time: {total_time / 60:.2f} minutes")

    #Track energy consumption
    tracker.stop()
    print(f"Epoch {epoch+1}/{epochs}, Energy consumption: {tracker}")

# Save the trained model (optional)
torch.save(model.state_dict(), "trained_model.pth")





[codecarbon ERROR @ 01:17:20] Error: Another instance of codecarbon is probably running as we find `/tmp/.codecarbon.lock`. Turn off the other instance to be able to run this one or use `allow_multiple_runs` or delete the file. Exiting.
[codecarbon WARNING @ 01:17:20] Another instance of codecarbon is already running. Exiting.


cpu


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Batch 1/1118 - Loss: 14.774067878723145
Batch 2/1118 - Loss: 13.566119194030762
Batch 3/1118 - Loss: 12.454340934753418
Batch 4/1118 - Loss: 10.46998405456543
Batch 5/1118 - Loss: 8.929386138916016
Batch 6/1118 - Loss: 7.495364189147949
Batch 7/1118 - Loss: 6.464634418487549
Batch 8/1118 - Loss: 5.809053897857666
Batch 9/1118 - Loss: 5.105774879455566
Batch 10/1118 - Loss: 4.957127094268799
Batch 11/1118 - Loss: 4.713294506072998
Batch 12/1118 - Loss: 4.262165546417236
Batch 13/1118 - Loss: 3.842937469482422
Batch 14/1118 - Loss: 3.935763120651245
Batch 15/1118 - Loss: 3.622844696044922
Batch 16/1118 - Loss: 3.5557310581207275
Batch 17/1118 - Loss: 3.213555097579956
Batch 18/1118 - Loss: 3.1771914958953857
Batch 19/1118 - Loss: 2.885312080383301
Batch 20/1118 - Loss: 3.0395069122314453
Batch 21/1118 - Loss: 2.9687654972076416
Batch 22/1118 - Loss: 2.921973466873169
Batch 23/1118 - Loss: 2.6426429748535156
Batch 24/1118 - Loss: 2.7179083824157715
Batch 25/1118 - Loss: 2.4686217308044434

[codecarbon WARNING @ 03:41:59] Another instance of codecarbon is already running. Exiting.


Batch 1118/1118 - Loss: 1.300163984298706
Epoch 1/1, Loss: 1.363289705542631
Epoch 1/1, Memory Usage: 25031.1796875 MB
Epoch 1/1, Total Training Time: 144.64 minutes
Epoch 1/1, Energy consumption: <codecarbon.emissions_tracker.EmissionsTracker object at 0x7adfe814e510>


In [3]:
#Test and evaluate

def generate_conversation(persona1, persona2):
    input_text = persona1 + persona2
    input_ids = TOKENIZER(input_text, return_tensors="pt").input_ids

    # Generate response
    output_ids = model.generate(input_ids, max_length=150, num_beams=5, temperature=0.7)
    output_text = TOKENIZER.decode(output_ids[0], skip_special_tokens=True)

    return output_text

persona1 = "I love hiking and nature walks. I also enjoy photography."
persona2 = "I prefer staying indoors and reading books. I love mystery novels."
generated_conversation = generate_conversation(persona1, persona2)

print("Generated Chatbot Conversation:\n", generated_conversation)

def evaluate(model, test_dataloader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
      for batch in test_dataloader:
            input_ids =  batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            # Pass labels as decoder_input_ids during evaluation
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            # Get predictions from the logits
            predictions = torch.argmax(outputs.logits, dim=2)
            # Compare predictions with shifted labels for accuracy calculation
            shifted_labels = labels[..., 1:]  # Shift labels to the right by one position
            predictions = predictions[..., :-1]  # Remove the last prediction
            # Calculate accuracy for valid predictions (ignoring padding)
            active_positions = shifted_labels != -100  # Ignore padding positions (-100)
            correct_predictions = (predictions == shifted_labels) * active_positions
            correct += correct_predictions.sum().item()
            total += active_positions.sum().item()
    print(f'Exactitude du modèle sur IMDb : {correct / total:.4f}')

evaluate(model, test_dataloader)

NameError: name 'TOKENIZER' is not defined

In [1]:
#metrics evaluation performance
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np

# Assuming you have predictions and true labels from your model
# Replace these with your actual predictions and true labels
predictions = predictions
true_labels = labels

# Calculate metrics
accuracy = accuracy_score(true_labels, predictions)
precision = precision_score(true_labels, predictions)
recall = recall_score(true_labels, predictions)
f1 = f1_score(true_labels, predictions)


# Create a DataFrame for logging
evaluation_data = {
    "Metric": ["Accuracy", "Precision", "Recall", "F1 Score"],
    "Value": [accuracy, precision, recall, f1]
    }
df = pd.DataFrame(evaluation_data)


# Display in a tab (Colab only)
from google.colab import data_table
data_table.DataTable(df)

#Alternative to display (works in notebooks as well)
#print(df)


NameError: name 'predictions' is not defined

In [ ]:
# prompt: compare the predictions with the actual output of the values from dataset

actual_outputs = [TOKENIZER.decode(item['labels'][0], skip_special_tokens=True) for item in test_dataset]

# Compare predictions with actual outputs
for i in range(len(predictions)):
    print(f"Prediction {i+1}: {predictions[i]}")
    print(f"Actual Output {i+1}: {actual_outputs[i]}")
    print("-" * 20)
